In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
# category_encoders, feature_tools, imblearn, tsfresh

/kaggle/input/criteo-dataset/dac/train.txt
/kaggle/input/criteo-dataset/dac/readme.txt
/kaggle/input/criteo-dataset/dac/test.txt


In [2]:
train_f_name = '/kaggle/input/criteo-dataset/dac/train.txt'
test_f_name = '/kaggle/input/criteo-dataset/dac/test.txt'

I_col_names = ['I{}'.format(i) for i in range(1, 14)]
C_col_names = ['C{}'.format(i) for i in range(1, 27)]

raw_tr_rows_cnt = 45840617 # rows in training set
raw_ts_rows_cnt = 6042135  # rows in testing set

In [3]:
from tqdm import tqdm_notebook
import collections
import gc
counts = [collections.defaultdict(int) for _ in range(26)]

'''
Comment by Runtong:
    I once tried to load source file as csv format but failed. The training set is toooooo large 
and pandas or csv module couldn't handle it(pd.read_csv or csv.DictReader). So, I used the most 
basic method to open the file and do parts of feature engineering based on that.   
'''

f = open(train_f_name,'r')
i = 0

ff = f.readlines()


In [4]:
for i,line in tqdm_notebook(enumerate(ff[:raw_tr_rows_cnt//2])):
    if i % 10 != 0: # Runtong: do downsampling, for time consumption concideration.
        continue
    row = line.strip('\n').split('\t')
    label = row[0]
    for j in range(1, 27):
        #field = 'C{0}'.format(j)
        #print(row[j - 27])
        if row[j - 27]:
            value = np.int32(int(row[j - 27],16))
            j -= 1
            counts[j][value] += 1
    del line
gc.collect()

f.close()

In [5]:
del ff
gc.collect()

0

In [6]:
freq_items = [collections.defaultdict(lambda : -1) for _ in range(26)]
Freq_Threshold = 5
# filter
for ind, dic in enumerate(counts[:26]):
    for k, c in dic.items():
        if c >= Freq_Threshold:
            freq_items[ind][k] = c
# counter
for ind, dic in enumerate(freq_items[:26]):
    for i, k in enumerate(dic.keys()):
        freq_items[ind][k] = i


In [7]:
list(zip(map(len,freq_items), map(len,counts)))

[(833, 1416),
 (530, 545),
 (25847, 738722),
 (27047, 252099),
 (207, 297),
 (14, 16),
 (10004, 11558),
 (408, 620),
 (3, 3),
 (17413, 42244),
 (4455, 5032),
 (26118, 648561),
 (3068, 3126),
 (26, 26),
 (7065, 10738),
 (26762, 481906),
 (10, 10),
 (3286, 4444),
 (1616, 1965),
 (3, 3),
 (26509, 579057),
 (11, 15),
 (15, 15),
 (16477, 67996),
 (60, 84),
 (13391, 47429)]

In [8]:
chunk_size = 4000000
train_pec = 0.5
train_pd = pd.read_csv(train_f_name,
                      sep = '\t',
                      names = ['label'] + I_col_names + C_col_names,
                      nrows = int(train_pec * raw_tr_rows_cnt),
                      dtype = dict(zip(I_col_names, [np.float32] * len(I_col_names))),
                      converters = dict(zip(C_col_names, [lambda x: int(x or 'ffffffff', 16)  for _ in C_col_names])),
                      iterator = True,
                      chunksize = chunk_size,
                      )

In [9]:
result = None
for tp in train_pd:
    tp[C_col_names] = tp[C_col_names].astype(np.int32)
    
    if not isinstance(tp, pd.DataFrame):
        result = tp
    else:
        result = pd.concat([result,tp], axis = 0)
train_pd = result

In [10]:
train_pd[C_col_names].head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,1761418852,-2132644709,-74227402,2068259780,633879704,2114768079,-562457160,529118562,-1489050352,-1462938364,...,-440764814,-180349255,568184265,-1322964323,129309004,-1,974593739,-976943996,-390581241,-1758995178
1,1761418852,-254869468,1869084645,1093094615,633879704,-26504475,-1842676544,185940084,-1489050352,726918651,...,130367684,-1337047440,568184265,1480633834,1626743326,-1,974593739,1139883659,-390581241,1931228757
2,679372879,173120604,47159414,-1047797375,633879704,2114768079,-947780447,185940084,-1489050352,990438539,...,-1895899521,873599373,-1,-1,-444087194,-1389337877,974593739,991444060,-1,-1
3,1761418852,739682630,-1448575384,773314294,633879704,-26504475,780822683,185940084,-1489050352,-269860037,...,512280399,1961833730,-1,-1,1798986918,-1,974593739,-1860721846,-1,-1
4,-1930399131,-1371102563,-938047301,-115150931,633879704,326208445,-1382047147,185940084,-1489050352,1384300855,...,512280399,649316263,-1,-1,566841706,-1,851920782,-1286655704,-1,-1


In [11]:
for i in tqdm_notebook(range(1, 27)):
    #print(i, 'train')
    train_pd['C{}'.format(i)] = train_pd['C{}'.format(i)].map(lambda x: freq_items[i-1][x]).astype(np.int32)
    #print(i, 'test')

In [12]:
train_pd.to_csv('./train_0_50.csv',index = False,header=False)

In [13]:
train_pd.head(20)

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,1.0,1.0,5.0,0.0,1382.0,4.0,15.0,2.0,181.0,...,0,0,0,0,-1,-1,0,0,0,0
1,0,2.0,0.0,44.0,1.0,102.0,8.0,2.0,2.0,4.0,...,5,19,0,1,94,-1,0,90,0,58
2,0,2.0,0.0,1.0,14.0,767.0,89.0,4.0,2.0,245.0,...,2,295,-1,-1,5,0,0,4,-1,-1
3,0,NaN,893.0,NaN,NaN,4392.0,NaN,0.0,0.0,0.0,...,4,74,-1,-1,-1,-1,0,37,-1,-1
4,0,3.0,-1.0,NaN,0.0,2.0,0.0,3.0,0.0,0.0,...,4,135,-1,-1,28,-1,1,8,-1,-1
5,0,NaN,-1.0,NaN,NaN,12824.0,NaN,0.0,0.0,6.0,...,1,453,-1,-1,564,3,3,68,-1,-1
6,0,NaN,1.0,2.0,NaN,3168.0,NaN,0.0,1.0,2.0,...,1,224,-1,-1,609,-1,11,24,-1,-1
7,1,1.0,4.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,74,-1,-1,-1,-1,1,37,-1,-1
8,0,NaN,44.0,4.0,8.0,19010.0,249.0,28.0,31.0,141.0,...,0,151,-1,-1,3,-1,1,4,-1,-1
9,0,NaN,35.0,NaN,1.0,33737.0,21.0,1.0,2.0,3.0,...,7,505,-1,-1,532,-1,1,531,-1,-1


In [14]:
'''
import lightgbm as lgb
lgb_train = lgb.Dataset(train_pd.drop('label', axis=1), 
                        train_pd['label'],
                        free_raw_data=True,
                       categorical_feature=C_col_names)
param = {'num_leaves': 10, 'num_trees': 50, 'objective': 'regression'}
gbm=lgb.train(param, lgb_train)
y_test = test_pd['label']
X_test = test_pd.drop('label', axis=1)
y_hat=gbm.predict(X_test)
from sklearn import metrics
test_auc = metrics.roc_auc_score(y_test,y_hat)
test_auc
'''

"\nimport lightgbm as lgb\nlgb_train = lgb.Dataset(train_pd.drop('label', axis=1), \n                        train_pd['label'],\n                        free_raw_data=True,\n                       categorical_feature=C_col_names)\nparam = {'num_leaves': 10, 'num_trees': 50, 'objective': 'regression'}\ngbm=lgb.train(param, lgb_train)\ny_test = test_pd['label']\nX_test = test_pd.drop('label', axis=1)\ny_hat=gbm.predict(X_test)\nfrom sklearn import metrics\ntest_auc = metrics.roc_auc_score(y_test,y_hat)\ntest_auc\n"